<h1>0. Packages</h1>

In [1]:
# 3rd party lib.
import numpy as np
import re
import skyfield.sgp4lib as spg4
import matplotlib 
import PyQt5
import chart_studio.plotly as py
import chart_studio
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import datetime as datetime
import os
from skyfield.api import wgs84
from enum import IntEnum    
from mpl_toolkits.mplot3d import Axes3D
from itertools import product, combinations
from matplotlib import cm
from scipy import integrate
from numba import njit, prange
from skyfield.api import load, EarthSatellite, wgs84

# User defined lib.
import generate_debris as gd
import CoordTransforms as ct
import visualize_deb as vis
import planetary_data

matplotlib.use('Agg')
%matplotlib qt5
debris_category = IntEnum('Category', 'rb sc soc')

<IPython.core.display.Javascript object>

In [2]:
# Create Directory to Save Figures to
if not os.path.exists("figures"):
    os.mkdir("figures")

# Create Directory to Save Interactive plots to
if not os.path.exists("plots"):
    os.mkdir("plots")
    
# Create Directory to Save gifs
if not os.path.exists("gifs"):
    os.mkdir("gifs")
    
# Retreiving API Keys from OS
PLOTLY_API_KEY = os.environ.get('PLOTLY_API_KEY')
PLOTYLY_USERNAME = 'rhumphreys2017'

# Log into chart studio for uploading plots
chart_studio.tools.set_credentials_file(username=PLOTYLY_USERNAME, api_key=PLOTLY_API_KEY)

<h1>1. Data Source</h1>

<h3>1.1 Loading TLE Data</h3>

In [3]:
# Opening the .txt file
with open("3le.txt") as f:
    txt = f.read()
    
sat_lines = re.findall('(.*?)\n(.*?)\n(.*?)\n', txt)

# Convert each group of 3 lines into a satelite object
def line_element_to_satellite(lines):
    title = lines[0]
    line1 = lines[1]
    line2 = lines[2]
    return spg4.EarthSatellite(line1, line2, name=title)

satellites = [line_element_to_satellite(lines) for lines in sat_lines]

<h3>1.2 Select orbital object for analysis</h3>

In [4]:
# Retrive test satellite from TLE Data
data = np.array(satellites)

# Getting the satellite
names = np.array([sat.name for sat in data])
i_oxp1 = np.argwhere(names == "0 OXP 1").flatten()[0]
satellite = data[i_oxp1]

satellite

<EarthSatellite 0 OXP 1 catalog #22489 epoch 2019-12-19 08:49:27 UTC>

# 2. Debris cloud simulation

## 2.1 Satellite breakup event

#### 2.1.1 Exploring the NASA Breakup model

In [ ]:
#
# Explosion 
#

m_target = 1000 # [kg]

# Do not need to specify projectile mass, and collision velocity for an explosion
# L_c [m], areas [m^2], masses [kg]
L_c, areas, masses, AM = gd.fragmentation(m_target, 0, 0, False, debris_category.rb, True)

# LC DATA
bins = [1e-3, 1e-2, 1e-1, 1, np.inf] # L_c bins in meters
h,b = np.histogram(L_c, bins=bins)

ch = [ { f'>{bins[i]}m':np.sum(h[i:]) for i in range(len(h))}]

df = pd.DataFrame(ch)

# MASS DATA
bins = [1, np.inf]
h,b = np.histogram(masses*1e3, bins=bins) # grams
ch = [ { f'>{bins[i]}g':np.sum(h[i:]) for i in range(len(h))}]

df2 = pd.DataFrame(ch)

# AREA DATA
bins = [1, np.inf]
h,b = np.histogram(areas*1e4, bins=bins) # cm^2
ch = [ { f'>{bins[i]}cm^2':np.sum(h[i:]) for i in range(len(h))}]

df3 = pd.DataFrame(ch)

# # VELOCITY DATA (A bit janky currently, always need to specify v_c event for explosions)
deltaV = 10**np.array(gd.distribution_deltaV(AM, 5, True)) # Returns as [km·s^-1]
bins = [1, np.inf]
h,b = np.histogram(deltaV, bins=bins)
ch = [ { f'>{bins[i]}km/s':np.sum(h[i:]) for i in range(len(h))}]
df4 = pd.DataFrame(ch)

results = pd.concat([df, df2, df3, df4], axis=1)
results
m_target = 1000 # [kg]

# Do not need to specify projectile mass, and collision velocity for an explosion
# L_c [m], areas [m^2], masses [kg]
L_c, areas, masses, AM = gd.fragmentation(m_target, 0, 0, False, debris_category.rb, True)

# LC DATA
bins = [1e-3, 1e-2, 1e-1, 1, np.inf] # L_c bins in meters
h,b = np.histogram(L_c, bins=bins)

ch = [ { f'>{bins[i]}m':np.sum(h[i:]) for i in range(len(h))}]

df = pd.DataFrame(ch)

# MASS DATA
bins = [1, np.inf]
h,b = np.histogram(masses*1e3, bins=bins) # grams
ch = [ { f'>{bins[i]}g':np.sum(h[i:]) for i in range(len(h))}]

df2 = pd.DataFrame(ch)

# AREA DATA
bins = [1, np.inf]
h,b = np.histogram(areas*1e4, bins=bins) # cm^2
ch = [ { f'>{bins[i]}cm^2':np.sum(h[i:]) for i in range(len(h))}]

df3 = pd.DataFrame(ch)

# # VELOCITY DATA (A bit janky currently, always need to specify v_c event for explosions)
deltaV = 10**np.array(gd.distribution_deltaV(AM, 5, True)) # Returns as [km·s^-1]
bins = [1, np.inf]
h,b = np.histogram(deltaV, bins=bins)
ch = [ { f'>{bins[i]}km/s':np.sum(h[i:]) for i in range(len(h))}]
df4 = pd.DataFrame(ch)

results = pd.concat([df, df2, df3, df4], axis=1)
results

In [ ]:
#
# Collision
#

m_target = 1000 # [kg]

# Do not need to specify projectile mass, and collision velocity for an explosion
# L_c [m], areas [m^2], masses [kg]
L_c, areas, masses, AM = gd.fragmentation(m_target, 10, 10, True, debris_category.rb, False)

# LC DATA
bins = [1e-3, 1e-2, 1e-1, 1, np.inf] # L_c bins in meters
h,b = np.histogram(L_c, bins=bins)

ch = [ { f'>{bins[i]}m':np.sum(h[i:]) for i in range(len(h))}]

df = pd.DataFrame(ch)

# MASS DATA
bins = [1, np.inf]
h,b = np.histogram(masses*1e3, bins=bins) # grams
ch = [ { f'>{bins[i]}g':np.sum(h[i:]) for i in range(len(h))}]

df2 = pd.DataFrame(ch)

# AREA DATA
bins = [1, np.inf]
h,b = np.histogram(areas*1e4, bins=bins) # cm^2
ch = [ { f'>{bins[i]}cm^2':np.sum(h[i:]) for i in range(len(h))}]

df3 = pd.DataFrame(ch)

# # VELOCITY DATA (A bit janky currently, always need to specify v_c event for explosions)
deltaV = 10**np.array(gd.distribution_deltaV(AM, 10, False)) # Returns as [km·s^-1]
deltaV *= 1e3
bins = [100, np.inf]
h,b = np.histogram(deltaV, bins=bins)
ch = [ { f'>{bins[i]}m/s':np.sum(h[i:]) for i in range(len(h))}]
df4 = pd.DataFrame(ch)

results = pd.concat([df, df2, df3, df4], axis=1)
results

#### 2.1.2 Visualizing the results of the NASA Breakup model

In [ ]:
# Validation Data
L_c, areas, masses, AM = gd.fragmentation(1000, 10, 10, True, debris_category.rb, False)
N_fragments_total = L_c.shape[0]
lambda_c = np.log10(L_c)

# Logarithmic Spaced Bins
def create_log_bins(values, nbins=100):
    #return np.geomspace(values.min(), values.max(), nbins)
    bins = np.geomspace(values.min(), values.max(), nbins)
    a = bins[1]/bins[0]
    bins = np.concatenate([[bins[0]/a], bins,[bins[-1]*a]])
    return bins


# Common Layout
layout = dict(
    autosize=False,
    width=500,
    height=500,
    template = 'plotly_white',
    yaxis = dict(
        range=[0,8e5],
        showexponent = 'all',
        exponentformat = 'e'
    ),
    legend=dict(
        y=0.5,
        traceorder='reversed',
        font=dict(
            size=16
        )
    )
)

###### Validating L_c ######

# Creating Histogram
h, b = np.histogram(L_c, bins=create_log_bins(L_c)) # Test data
# Make figure
fig = go.Figure(data=[go.Scatter(x=b, y=h, mode='lines', hoverinfo='all',
                                 line=dict(shape='hvh'))],
                layout=layout)

# Update figure
fig.update_xaxes(type="log")
fig.update_layout(
    title = 'Characteristic Length Distribution',
    xaxis_title=r'$\log_{10}(L_{c}\:[m])$',
    yaxis_title=r'$N_f$'
)

# Plot
fig.write_image("figures/N_f_vs_L_c.png", width=500, height=500, scale=2)
py.iplot(fig, filename="Characteristic Length Distribution")

# # # ###### Validating Areas ######

# Creating Histogram
h, b = np.histogram(areas, bins=create_log_bins(areas))

# Make figure
fig2 = go.Figure(data=[go.Scatter(x=b, y=h, mode='lines', hoverinfo='all',
                                 line=dict(shape='hvh'))],
                layout=layout)

# Update figure
fig2.update_xaxes(type="log")
fig2.update_layout(
    title = 'Area distribution',
    xaxis_title=r'$\log_{10}(A\:[m^2])$',
    yaxis_title=r'$N_f$',
)

# Plot
fig2.write_image("figures/N_f_vs_A.png", width=500, height=500, scale=2)
py.iplot(fig2, filename="Area distribution")

# ###### Validating MASS ######

# # Creating Histogram
h, b = np.histogram(masses, bins=create_log_bins(masses))

# Make figure
fig3 = go.Figure(data=[go.Scatter(x=b, y=h, mode='lines', hoverinfo='all',
                                 line=dict(shape='hvh'))],
                layout=layout)

# Update figure
fig3.update_xaxes(type="log")
fig3.update_layout(
    title = 'Mass Distribution',
    xaxis_title=r'$\log_{10}(M\:[kg])$',
    yaxis_title=r'$N_f$'
)

# Plot
fig3.write_image("figures/N_f_vs_M.png", width=500, height=500, scale=2)
py.iplot(fig3, filename="Mass Distribution")

### Validating Velocity
deltaV = np.array(gd.distribution_deltaV(AM, 10, False))
# Creating Histogram
h, b = np.histogram(deltaV, bins=create_log_bins(deltaV))

# Make figure
fig4 = go.Figure(data=[go.Scatter(x=b, y=h, mode='lines', hoverinfo='all',
                                 line=dict(shape='hvh'))],
                layout=layout)

# Update figure
fig4.update_xaxes(type="log")
fig4.update_layout(
    title = 'Velocity distribution',
    xaxis_title=r'$\log_{10}(V\:[ms^-1])$',
    yaxis_title=r'$N_f$',
)

# Plot
fig4.write_image("figures/N_f_vs_V.png", width=500, height=500, scale=2)
py.iplot(fig4, filename="Velocity distribution")

#### 2.1.2 Perform fragmentation event

In [5]:
# Performing fragmentation event
ts = load.timescale(builtin=True)
t_fragmentation = ts.now()
geocentric      = satellite.at(t_fragmentation)
init_position   = geocentric.position.m

m_target         = 250   # [kg] (Approx. mass of starlink sat)
m_projectile     = 200     # [kg]
v_impact         = 2   # [km·s^-1] (Measured relative to the target) (Needs to be in km·s^-1)
is_catastrophic  = False
is_explosion     = False

L_c, areas, masses, AM = gd.fragmentation(m_target, m_projectile, v_impact, is_catastrophic, debris_category.sc, is_explosion)
deltaV = np.array(gd.distribution_deltaV(AM, v_impact, False)) # Returns as [km·s^-1]
deltaV = deltaV * 1e3    #[m·s^-1]

/Users/reecehumphreys/Developer/Thesis/Thesis/generate_debris.py:398: RuntimeWarning: invalid value encountered in power
  found_L_c = np.sort((area / 0.556945)**(1/2.0047077)) # Inversing the Area function defined above


#### 2.1.3 Retriving Cartesian coordinates of debris fragments

In [6]:
from numpy.linalg import norm
init_position = geocentric.position.m
deb_positions       = np.empty((len(AM), 3))
deb_positions[:, :] = init_position[None,:]
deb_velocities      = gd.velocity_vectors(len(AM), geocentric.velocity.m_per_s, deltaV)

#### 2.1.4 Converting coordinates to Keplerian elements

In [7]:
keplerian_state     = ct.rv2coe(deb_positions, deb_velocities, planetary_data.earth['mu'])

# Removing fragments that would renter earth
periapsis           = keplerian_state[0, :] * (1 - keplerian_state[1, :])
I                   = np.argwhere(periapsis > planetary_data.earth['radius'])
keplerian_state     = np.squeeze(keplerian_state[:, I])
areas               = areas[I].flatten()  # When doing the indexing , a 1d dim being added which is unneccesary
masses              = masses[I].flatten() # When doing the indexing , a 1d dim being added which is unneccesary

## 2.2 Ring Formation

In [8]:
## Shrink number of debris being used
indexes = np.random.default_rng().choice(keplerian_state.shape[1], size=1000, replace=False)

ks = keplerian_state[:, indexes]
masses = masses[indexes]
areas = areas[indexes]
deb_positions = deb_positions[indexes, :]
deb_velocities = deb_velocities[indexes, :]

In [9]:
import Perturbations as OP
import planetary_data as pd
from Perturbations import null_perts

# Cleanup states to remove any fragments that would deorbit, given no perturbations
periapsis     = ks[0, :] * (1 - ks[1, :])
I             = np.argwhere(periapsis > pd.earth['radius'])
ks_pruned     = np.squeeze(ks[:, I])
T             = ks_pruned[8, :]
areas_pruned  = areas[I].flatten()  # When doing the indexing , a 1d dim being added which is unneccesary
masses_pruned = masses[I].flatten() # When doing the indexing , a 1d dim being added which is unneccesary

# Propagate orbit for a period of time
perts = null_perts()
perts['aero'] = True
perts['J2']   = True
op = OP.OrbitPropagator(ks_pruned, areas_pruned, masses_pruned, [0, 1000*np.ceil(max(T))], 60*30, perts=perts)
op.propagate_orbit()

# Get the cartesian state representation
cartesian_states = op.cartesian_representation()

<h4>3.4.2 Particle Debris flux</h4>

Using a particle flux to determine when the fragments of the debris have finished the formation of the ring. Indicating the end of the first phase of the debris cloud formation. This is accomplished by creating an xz plane and detecting when particles have switched from one side to the other. This approach will cause a peak as fragments pass through that becomes uniform as the debris becomes uniformly spread out.

<h4>3.4.3 Convergence of the flux</h4>

The next step is determining when the fragments have ended the torroid formation phase. This occurs when the fragments are approximately uniformally spread out. We can check to see when the flux meets a convergence criterion to determine when this happens.

Now that the band has formed, we can shift away from propagating the exact position of each fragments and inplace propgate their changes in eccentricity and semi major axis due to drag. To do this first we must get the final states of the debris after the band has formed.

In [10]:
import time
import matplotlib.dates as mdates
from dateutil import tz

def fragmentation_flux(X):
    return np.sum((X[:-1, :, 1] < 0) & (X[1:, :, 1] > 0), axis=1)
    
position = cartesian_states[:, 0, :, :]
flux = fragmentation_flux(position)

w = 100 # Window of points to look at
tol = 5
convergence_ratio = np.array([np.var(flux[i:i+w])/np.mean(flux[i:i+w]) for i in range(len(flux))])    
intersection_index = np.argwhere(convergence_ratio <= tol).flatten()[0]

# datetimes
t_flux = t_fragmentation.utc_datetime() + np.array(range(len(flux))) * datetime.timedelta(minutes = 5)

# Removing last window from `t_flux`, `flux`, and `convergence_ratio` bc. not well defined for last values
t_flux = t_flux[:-w]
flux = flux[:-w]
convergence_ratio = convergence_ratio[:-w]

# Pruning data to the end of the ring formation
cs_toroid = cartesian_states[:intersection_index, :, :, :]
ks_toroid = op.states[0:intersection_index, :, :]
op.states = ks_toroid

## 2.3 Band Formation

### 2.3.1 Drag Implementation

In [ ]:
import Aerodynamics as aero

upper_bound = 900 * 1e3                         #[m]
altitudes   = np.arange(0, upper_bound, 25)      #[m]
rho         = aero.atmosphere_density(altitudes) #[kg·m^-3]

I_standard = np.argwhere(altitudes/1e3 == 25).flatten()[0]
I_cira    = np.argwhere(altitudes/1e3 == 500).flatten()[0]

# Plotting the Exponential Atmospheric Model

layout = go.Layout(
    title        = go.layout.Title(text='Altitude (z) vs. Atmospheric Density (ρ)',
                                   x=0.5),
    xaxis_title  = 'z [km]',
    yaxis_title  = '$\log_{10}(\\rho\:[kg·m^{-3}])$',
    template     = 'plotly_white',
    legend       = go.layout.Legend(yanchor="top",
                             y=0.99,
                             xanchor="right",
                             x=0.99)
)

data = [
    go.Scatter(x=altitudes[:I_standard] / 1e3, y=rho[:I_standard],
                    mode='lines',
                    name='U.S Standard Atmosphere'),
    go.Scatter(x=altitudes[I_standard:I_cira] / 1e3, y=rho[I_standard:I_cira],
                    mode='lines',
                    name='CIRA-72'),
    go.Scatter(x=altitudes[I_cira:] / 1e3, y=rho[I_cira:],
                    mode='lines',
                    name='CIRA-72 with T_infinity = 1000K')
]

fig = go.Figure(data=data, layout=layout)
fig.update_yaxes(type="log")


fig.write_image("figures/Atmospheric_Density_v_Altitude.png", width=500, height=500, scale=2)
f2 = go.FigureWidget(fig)
f2

### 2.3.1 Applying Perturbations to Satellite

In [ ]:
from scipy.special import iv
from scipy import integrate
import Aerodynamics as aero

op.tspan[-1] = 3600*24*365*3
op.dt = 3600*24
de, da, di, dOmega, domega, dnu, dp = op.propagate_perturbations()

# 3. Analysis

## 3.1 Flux

### 3.1.1 FLux plot

In [11]:
# Creating Flux v. Time plot
layout = go.Layout(
    title        = dict(text='$\\text{Flux}\:(\\Phi)\:\\text{vs. Time }(t)$',
                        x=0.5),
    xaxis_title  = '$t\:[days]$',
    yaxis_title  = '$\\text{ Number of fragments passing XZ plane, }\Phi\:$',
    template     = 'plotly_white'
)


data = [
    go.Scatter(x=t_flux, y=flux,
               mode='lines',
               name='Flux'),
    go.Scatter(x=[t_flux[intersection_index], t_flux[intersection_index]], y=[0, np.max(flux)],
               mode='lines',
               line=dict(dash = 'dash'),
               name='Convergence')
]

fig1 = go.Figure(data=data, layout=layout)

# Stopping data to have half before intersection index and half after
index_stop = intersection_index * 2
if index_stop > len(flux) - 1 : index_stop = len(flux) - 1
fig1.update_layout(xaxis_range=[t_flux[0],t_flux[index_stop]])

# Saving plot as an image and uploading it to plotly
fig1.write_image("figures/Flux_v_Time.png", width=500, height=500, scale=2)
#py.iplot(fig1, filename="Flux v. Time")

2546


### 3.1.2 Convergence Ratio plot

In [12]:
#Creating Convergence Ratio v. Time plot
layout = go.Layout(
    title        = dict(text='Convergence ratio vs. Time (t)',
                        x=0.5),
    xaxis_title  = '$t\:[days]$',
    yaxis_title  = 'Convergence ratio []',
    template     = 'plotly_white',
    legend       = go.layout.Legend(yanchor="top",
                             y=0.99,
                             xanchor="right",
                             x=0.99)
)
data = [
    go.Scatter(x=t_flux, y=convergence_ratio,
               mode='lines',
               name='Convergence ratio'),
    go.Scatter(x=[t_flux[intersection_index], t_flux[intersection_index]], y=[0, np.max(flux)],
               mode='lines',
               line=dict(dash = 'dash'),
               name='Convergence time'),
    go.Scatter(x=[t_flux[0], t_flux[-1]], y=[tol, tol],
               mode='lines',
               line=dict(dash = 'dash'),
               name='Tolerance'),
]
fig2 = go.Figure(data=data, layout=layout)
fig2.update_yaxes(type="log")
fig2.write_image("figures/Convergence_Ratio_v_Time.png", width=500, height=500, scale=2)
#py.iplot(fig2, filename="Convergence Ratio v. Time")

## 3.2 Ring visualization

In [ ]:
import plotly.express as px
import plotly.io as pio
import pandas

spherical_earth_map = np.load('map_sphere.npy') 

pos_toroid = cs_toroid[:, 0, :, :]/1e3
N_timesteps = pos_toroid.shape[0]
N_fragments = pos_toroid.shape[1]
r_E = op.cb['radius'] / 1e3
xm, ym, zm = spherical_earth_map.T * r_E

# Converting data to pandas dataframe
df = pandas.DataFrame()
# *** Update this if chnage timestep in initial orbit propagation ***
dt = 60 * 5 #[s]
# Want to show the evolution in 30 min
timesteps = np.arange(0,N_timesteps, 6)

for t in timesteps:   
    step = t*np.ones_like(N_timesteps)
    time = dt * step / 60 #[min]
    d = {'X': pos_toroid[t, :, 0],
         'Y': pos_toroid[t, :, 1],
         'Z':pos_toroid[t, :, 2],
         'Min.': time,
         'a': ks_toroid[t, 0, :]/1e3,
         'e': ks_toroid[t, 1, :],
         'i': ks_toroid[t, 2, :],
        }
    df = pandas.concat([df, pandas.DataFrame(data=d)])

# Creating visual
def spheres(size, clr, dist=0): 

    # Set up 100 points. First, do angles
    theta = np.linspace(0,2*np.pi,100)
    phi = np.linspace(0,np.pi,100)

    # Set up coordinates for points on the sphere
    x0 = dist + size * np.outer(np.cos(theta),np.sin(phi))
    y0 = size * np.outer(np.sin(theta),np.sin(phi))
    z0 = size * np.outer(np.ones(100),np.cos(phi))

    # Set up trace
    trace= go.Surface(x=x0, y=y0, z=z0, colorscale=[[0,clr], [1,clr]])
    trace.update(showscale=False)

    return trace

fig = px.scatter_3d(
    data_frame=df,
    x='X',
    y='Y',
    z='Z',
    title='Evolution of debris cloud to toroid formation',
    hover_data={'Min.': False, 'X': False, 'Y':False, 'Z':False, 'a':':.1f', 'e':':.4f','i':':.1f' },
    height=800,                 # height of graph in pixels
    width =800,
    animation_frame='Min.',   # assign marks to animation frames
    range_x=[-r_E - 1000,r_E + 1000],
    range_z=[-r_E - 1000,r_E + 1000],
    range_y=[-r_E - 1000,r_E + 1000],

)
fig.update_traces(marker={'size': 3})
# Add Earth
earth=spheres(r_E, '#F0FFFF', 0) # Earth
#fig.add_trace(go.Scatter3d(x=xm, y=ym, z=zm, mode='lines', line=dict(color=zm, colorscale='Viridis')))
fig['layout']['scene']['aspectmode'] = 'cube'
fig.add_trace(earth)
fig.update_layout(transition = {'duration': 2000})
fig.write_html("plots/ring.html")

## 3.3 Band visualization

In [ ]:
temp = np.zeros_like(da) # The params set to 0 dont matter for converting to rv
ks_propagated = np.swapaxes(np.stack([da, de, di, dOmega, domega, temp, dnu, dp, temp, temp]).T, 1, 2)
ks_final = np.concatenate([ks_toroid, ks_propagated])
op.states = ks_final
cs_final = op.cartesian_representation()

In [ ]:
import pandas as pandas
import plotly.express as px

pos_toroid = cs_final[cs_toroid.shape[0]-1:, 0, :, :]/1e3
N_timesteps = pos_toroid.shape[0]
N_fragments = pos_toroid.shape[1]
r_E = op.cb['radius'] / 1e3


# Converting data to pandas dataframe
df = pandas.DataFrame()
# *** Update this if chnage timestep in initial orbit propagation ***
dt = 60 * 5 #[s]
# Want to show the evolution in 1 day steps
timesteps = np.arange(0,N_timesteps, 5)

for t in timesteps:   
    step = t*np.ones_like(N_timesteps)
    time = step  #[day]
    d = {'X': pos_toroid[t, :, 0],
         'Y': pos_toroid[t, :, 1],
         'Z':pos_toroid[t, :, 2],
         'Day': time,
        }
    df = pandas.concat([df, pandas.DataFrame(data=d)])

def spheres(size, clr, dist=0): 

    # Set up 100 points. First, do angles
    theta = np.linspace(0,2*np.pi,100)
    phi = np.linspace(0,np.pi,100)

    # Set up coordinates for points on the sphere
    x0 = dist + size * np.outer(np.cos(theta),np.sin(phi))
    y0 = size * np.outer(np.sin(theta),np.sin(phi))
    z0 = size * np.outer(np.ones(100),np.cos(phi))

    # Set up trace
    trace= go.Surface(x=x0, y=y0, z=z0, colorscale=[[0,clr], [1,clr]])
    trace.update(showscale=False)

    return trace
fig = px.scatter_3d(
    data_frame=df,
    x='X',
    y='Y',
    z='Z',
    title='Evolution of debris cloud to Band formation',
    #labels={'Years in school (avg)': 'Years Women are in School'},
    #hover_data={'Min.': False, 'X': False, 'Y':False, 'Z':False, 'a':':.1f', 'e':':.4f','i':':.1f' },
    #hover_name='Orbital Elements',        # values appear in bold in the hover tooltip
    height=800,                 # height of graph in pixels
    width =800,
    animation_frame='Day',   # assign marks to animation frames
    range_x=[-r_E - 1000,r_E + 1000],
    range_z=[-r_E - 1000,r_E + 1000],
    range_y=[-r_E - 1000,r_E + 1000],

)
fig.update_traces(marker={'size': 1.5, 'color':'#6372f4'})
# Add Earth
earth=spheres(r_E, '#ffffff', 0) # Earth
fig.add_trace(earth)
#fig.add_trace(go.Scatter3d(x=xm, y=ym, z=zm, mode='lines', line=dict(color=zm, colorscale='Viridis')))
fig['layout']['scene']['aspectmode'] = 'cube'
fig.update_layout(transition = {'duration': 2000})
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)')
fig.write_html("plots/band.html")

## 3.4 Time to deorbit 

In [ ]:
import plotly.figure_factory as ff
import matplotlib.cm as cm

AM = op.A / op.M
z  = (da * (1 - de)) - op.cb['radius']
z[z < 100*1e3] = 0

layout = go.Layout(
    title        = dict(text='Altitude of 50 debris fragments over 3 years',
                        x=0.5),
    xaxis_title  = '$t\:[days]$',
    yaxis_title  = 'Altitude [km]',
    template     = 'plotly_white',
    legend       = go.layout.Legend(yanchor="top",
                             y=0.99,
                             xanchor="right",
                             x=0.99)
)

data = []

for i in range(25):
    alt = np.trim_zeros(z[i, :]) / 1e3
    scatter = go.Scatter(x=[i for i in range(len(alt))], y=alt,
               mode='lines')
    data.append(scatter)
    
fig = go.Figure(data=data, layout=layout)
fig.update_layout(coloraxis=dict(colorscale='RdBu'), showlegend=False)
fig.show()    
fig.write_image("figures/oxp_altitudes.png", width=500, height=500, scale=2)

## 3.5 Debris spread

In [ ]:
index = int(np.ceil(ks_propagated.shape[0]*.10)) # index near begining
raan_0 = ks_propagated[index, 3, :].copy() % 360
raan_0[raan_0 > 180] -= 360 # Converting angles to new range

raan_mid = ks_propagated[ks_propagated.shape[0] // 2, 3, :].copy()  % 360
raan_mid[raan_mid  > 180] -= 360 

raan_f = ks_propagated[-1, 3, :].copy() % 360
raan_f[raan_f > 180] -= 360

In [ ]:
import math
import plotly.figure_factory as ff

uniform_dist = np.random.uniform(-180, 180, len(raan_0))
group_labels = ['$\Omega_{initial}$', '$\Omega_{midpoint}$', '$\Omega_{final}$', 'uniform']
fig = ff.create_distplot([raan_0, raan_mid, raan_f, uniform_dist], group_labels, show_hist =  False)

# Updating the uniform curve to be dashed
index = np.argwhere(np.array([data.legendgroup for data in fig.data]) == 'uniform')[0][0]
fig.data[index].line = dict(color='red', width=2,
                             dash='dash')

# Layout
fig.layout['title'] = dict(text='Longitude of the ascending node distribution',
                        x=0.5)
fig.layout['xaxis_title'] = '$\Omega\:[deg]$'
fig.layout['yaxis_title'] = 'Kernel density estimation'
fig.layout['template'] = 'plotly_white'


fig.write_image("figures/oxp_dist.png", width=500, height=500, scale=2)
py.iplot(fig, filename="Longitude of the ascending node distribution")

# 4. Conclusion